In [15]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import numpy as np
import time
import os
import asyncio, aiohttp, nest_asyncio
from aiohttp_retry import RetryClient, ExponentialRetry
nest_asyncio.apply()
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

trailing_num_days = 90

chains_tg = [
        'Ethereum'
        ,'Optimism'
        ,'Binance'
        ,'Arbitrum'
        ,'Polygon'
        ,'Avalanche'
        ]

In [16]:
#https://stackoverflow.com/questions/23267409/how-to-implement-retry-mechanism-into-python-requests-library

import logging
import requests

from requests.adapters import HTTPAdapter, Retry

# logging.basicConfig(level=logging.DEBUG)

s = requests.Session()
retries = Retry(total=10, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

pwd = os.getcwd()
if 'L2 TVL' in pwd:
    prepend = ''
else:
    prepend = 'L2 TVL/'

In [17]:
all_api = 'https://api.llama.fi/charts/'
dflist = []
for i in chains_tg:
        print(i)
        res = pd.DataFrame( r.get(all_api + str(i), headers=header).json() )
        res['chain'] = str(i)
        res['date'] = pd.to_datetime(res['date'], unit ='s') #convert to days
        res['rank_desc'] = res['date'].rank(method='dense',ascending=False).astype(int)
        res = res.sort_values(by='date',ascending=True)
        res['t90d_totalLiquidityUSD'] = res['totalLiquidityUSD'].shift(90)
        dflist.append(res)
dfs = pd.concat(dflist)

Ethereum
Optimism
Binance
Arbitrum
Polygon
Avalanche


,date,totalLiquidityUSD,chain,rank_desc
0,2020-12-20,0.000000e+00,Avalanche,625
1,2021-02-03,1.244221e+02,Avalanche,624
2,2021-02-04,3.612846e+06,Avalanche,623
3,2021-02-05,4.860423e+06,Avalanche,622
4,2021-02-06,6.220490e+06,Avalanche,621
...,...,...,...,...
620,2022-10-15,1.686233e+09,Avalanche,5
621,2022-10-16,1.677324e+09,Avalanche,4
622,2022-10-17,1.685730e+09,Avalanche,3
623,2022-10-18,1.703840e+09,Avalanche,2


In [ ]:
display()